## Download dataset
I've found that I've had some problems with unzip/shutil on this file. I'd suggest using 7zip to download it. You'll get an error that looks like this, but it'll still work.
```
ERRORS:
Headers Error

--
Path = /home/stephen/ds1/ds1.zip
Type = zip
ERRORS:
Headers Error
Physical Size = 11582983320
64-bit = +
Archives with Errors: 1
```
You will need to install 7zip command line for the unzipping to work, on ubuntu this can be done with ``apt install 7z``

In [ ]:
from subprocess import Popen, PIPE, STDOUT
import wget
from pathlib import Path
path_to_data = "data/Dataset 1 (Simplex)/"
if not Path(path_to_data).exists():
    wget.download('http://staff.ee.sun.ac.za/mjbooysen/Potholes/Slow/Dataset 1 (Simplex).zip',out='dataset.zip')
    out = Popen("7z x dataset.zip -odata",shell=True,stdin=PIPE, stdout=PIPE, stderr=STDOUT)

Allow matplotlib to render inside the notebook

In [ ]:
%matplotlib inline 

Required dependencies

In [ ]:
import requests
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import mobilenet_v2
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets.folder import ImageFolder
from torch.optim import adam
from torch.optim import lr_scheduler
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from os import listdir
import numpy as np

Load the pretrained mobilenet model from [torchivision.models](https://pytorch.org/docs/stable/torchvision/models.html)

As the name implies, this is a much lighter weight convolutional neural network designed to be run on devices with limited compute capacity.

In [ ]:
model = mobilenet_v2(pretrained=True)

Use CUDA to offload training to the GPU if possible, otherwise use CPU (Will be significantly slower if training on a CPU)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
model.cuda(device=device)
print(f"Using device {device}")

I'm going to be using the [ImageFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder) 
dataloader to automatically create the correct inputs and classes, in this case the two classes it creates are
 ``positive`` and ``negative`` based on

In [ ]:
from pathlib import Path
train_path = Path(str(path_to_data) + "/Train data")
listdir(train_path.absolute())

Data Augmentation is a good way to increase the training accuracy of a dataset [Table 3, Wang, Perez 2017](http://cs231n.stanford.edu/reports/2017/pdfs/300.pdf)

Torchivision gives us some easy to use transforms to augment our training images.

In [ ]:
transform = transforms.Compose([
    transforms.Resize([512,512]),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=[0.8,1.2],contrast=[0.8,1.2],saturation=[1,2],hue=[-0.2,0.2]),
    transforms.ToTensor()
    ])

In [ ]:
# Create the test/train datasets. Will split this in another step.
train_data = ImageFolder(root=train_path.absolute(),transform=transform)
test_data = ImageFolder(root=train_path.absolute(),transform=transform)
# Split this into a test and train set
samples_test,samples_train,targets_test,targets_train = train_test_split(train_data.samples,train_data.targets,test_size = .8,random_state=42,stratify=train_data.targets)
test_data.targets = targets_test
train_data.targets = targets_train
test_data.samples = samples_test
train_data.samples = samples_train
# Turn these datasets into DataLoaders (generators)
test_data_loader = DataLoader(test_data,batch_size=12,shuffle=True,num_workers=0,)

train_data_loader = DataLoader(train_data,batch_size=12,shuffle=True,num_workers=0)
print(f"Using classes {test_data.classes}")
print(f"There are {len(train_data.samples)} items in training data")
print(f"There are {len(test_data.samples)} items in validation data")

The current model assumes you're going to have a lot more outputs as it was predicting 1000 classes as part of ImagNet. 
We're going to chop this classifier layer off and put our own one

In [ ]:
print("Existing classifier")
print(model.classifier )
model.classifier = nn.Sequential(nn.Dropout(p=0.2,inplace=True),
                                 nn.Linear(in_features=1280,out_features=len(test_data.classes)))
print("New classifier")
print(model.classifier)


For the first part of training, were going to freeze a majority of the layer weights and only train the classifier initially

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
for param in model.classifier.parameters():
    param.requires_grad = True

using cross entroy as a loss function since this is a binary classification problem

In [ ]:
criteria = nn.CrossEntropyLoss()
model.train()
torch.cuda.set_device(device)
# Suppress model output
_ = model.cuda(device=device)

Creating a generic training function

In [ ]:
def run_epoch(model, data_loader, train,optimizer,lr_optim):
    losses = []
    if train:
        model.train()
    else:
        model.eval()
    for input,label in tqdm(data_loader):
        in_dat = input.to(device=device, dtype=torch.float)
        label = label.to(device=device, dtype=torch.long)
        if train:
            optimizer.zero_grad()
            outputs = model(in_dat)
            loss_val = criteria(outputs,label)
            losses.append(loss_val.item())
            loss_val.backward()
            optimizer.step()
            lr_optim.step()
        else:
            outputs = model(in_dat)
            loss_val = criteria(outputs,label)
            losses.append(loss_val.item())
    return losses[0]


def train(train_dl, test_dl,model,learning_rate,epoches):
    train_losses = []
    valid_losses = []
    
    optimizer = adam.Adam(model.parameters(),lr=learning_rate)
    lr_optim = lr_scheduler.CosineAnnealingLR(optimizer,len(train_dl))
    for epoch in range(0,epoches):
        trn_loss = run_epoch(model, train_dl,train=True,optimizer=optimizer,lr_optim=lr_optim)
        val_loss = run_epoch(model, test_dl,train=False,optimizer=optimizer,lr_optim=lr_optim)
        # return averages of the epoch
        train_losses.append(np.average(trn_loss)) 
        valid_losses.append(np.average(val_loss))
    plt.clf()
    plt.plot(train_losses)
    plt.plot(valid_losses)
    plt.legend(['training losses','validation losses'])
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and validation loss of model")
    plt.show()
    return train_losses, valid_losses

In [ ]:
t_loss, v_loss = train(train_data_loader,test_data_loader,model,learning_rate=0.001,epoches=10)


In [ ]:
torch.save(model,'firststep.model')

In [ ]:
# Take approx the last half of the parameters and set them to trainable
param_count = 0
for param in model.parameters():
    if param_count > 130:
        param.requires_grad = True
t_loss, v_loss = train(train_data_loader,test_data_loader,model,learning_rate=0.0001,epoches=10)  
torch.save(model,'secondstep.model')

In [ ]:
# Train every parameter but with a very low learning rate
for param in model.parameters():
    param.requires_grad = True
t_loss, v_loss = train(train_data_loader,test_data_loader,model,learning_rate=0.00001,epoches=6)
torch.save(model,'thirdstep.model')
